In [1]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - Hugging Face Transformers의 Whisper 파이프라인으로 STT(음성→텍스트)를 수행합니다.
#  - 오디오 파일을 로드/리샘플링/전처리하여 모델 입력 형식으로 변환합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: '%pip install --upgrade pip '
# ========================= 코드 해설 주석 끝 ===========================
%pip install --upgrade pip 
%pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of datasets[audio] to determine which version is compatible with other requirements. This could take a while.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - GPU(CUDA) 사용 여부/장치 설정 및 메모리 최적화를 수행합니다.
#  - 오디오 파일을 로드/리샘플링/전처리하여 모델 입력 형식으로 변환합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: '%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126'
# ========================= 코드 해설 주석 끝 ===========================
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


In [1]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - Hugging Face Transformers의 Whisper 파이프라인으로 STT(음성→텍스트)를 수행합니다.
#  - Whisper 계열 모델/프로세서를 로드하고, 디코딩 옵션을 설정합니다.
#  - GPU(CUDA) 사용 여부/장치 설정 및 메모리 최적화를 수행합니다.
#  - 오디오 파일을 로드/리샘플링/전처리하여 모델 입력 형식으로 변환합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: '# huggingface_whisper-tiny.ipynb'
# ========================= 코드 해설 주석 끝 ===========================
# huggingface_whisper-tiny.ipynb
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from datasets import load_dataset
import torchaudio


# ✅ 1. 디바이스 및 데이터 타입 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# ✅ 2. 모델 로드 (Whisper-tiny)
model_id = "openai/whisper-tiny"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
)
model.to(device)

# ✅ 3. 예제 오디오 데이터셋 로드
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

# ✅ 4. 오디오 입력 전처리
inputs = processor(
    sample["array"],
    sampling_rate=sample["sampling_rate"],
    return_tensors="pt"
).to(device, torch_dtype)

# ✅ 5. 텍스트 생성
generated_ids = model.generate(inputs.input_features)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("🗣️ 인식 결과:", transcription)


d:\leehyoungyeol\python_ai_agent\myvenv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

🗣️ 인식 결과:  Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Latins' work is really Greek after all, and can discover
